# Projeto 1 - Ciência dos Dados

Nome: Cesar Ades

Nome: Jonathan Sutton

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [35]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [36]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o trabalho!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com o trabalho!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [37]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,só queria o meu ps4 😭,0.0
1,eu que comprei no pc e ps4: 🤡🤡🤡 https://t.co/f...,0.0
2,@biceta_rj e eu que só peguei o ps4 em 2019???...,1.0
3,@its_iucas minha herança: desgosto e um ps4,0.0
4,@duduxth1 fall guys eu tenho de graça no ps4,1.0


In [38]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,0
1,se isso influenciasse no preço do ps4/ps5 eu a...,1
2,quem ai quer me dar um ps4 ?,0
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,0
4,@dudalbessa falando em tv\neu queria comprar u...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Nosso produto é o Play Station 4 que é um console para videogames fabricado pela Sony, tem diversos jogos e utiliza controles bluetooth.Consideramos tweets relevantes aqueles que envolviam informações sobre o preço, local de fabricação, performance do produto, jogos que podem ser usados, lançamentos de jogos e falhas que ocorrem com o console e e os produtos relacionados a ele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [39]:
#Definindo as variáveis como categóricas
train['Relevância'] = train['Relevância'].astype('category')
test['Relevância'] = test['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Não relevante','Relevante']  
test['Relevância'].cat.categories = ['Não relevante','Relevante']

In [40]:
test.head(5)

,Teste,Relevância
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante
2,quem ai quer me dar um ps4 ?,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante


In [41]:
def separate_emoji(text): #separa emojis dos outros caracteres
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

In [42]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', separate_emoji(text))
    return text_subbed    

Na célula abaixo, fizemos o percentual de tweets relevantes e irrelevantes na parte de treinamentos.

In [89]:
def treinamento(data):
    p_relevantes = dict()
    p_irrelevantes = dict()
    p_possiveis = set()
    for index,row in data.iterrows():
        tweet = row.Treinamento
        for palavra in cleanup(tweet).split():
            p_possiveis.add(palavra)
            if row.Relevância == 'Relevante':
                p_relevantes[palavra] = p_relevantes.get(palavra,0)+1
            else:
                p_irrelevantes[palavra] = p_irrelevantes.get(palavra,0)+1
    return (p_possiveis,p_relevantes,p_irrelevantes)
#p_possiveis,p_relevantes,p_irrelevantes = treinamento(train)

In [90]:
p_count = treinamento(train)

In [91]:
resultados_treino = train.Relevância.value_counts(True)
relevantes_treino = resultados_treino[0]
irrelevantes_treino = resultados_treino[1]
resultados_brutos_treino = train.Relevância.value_counts()
relevantes_brutos_treino = resultados_brutos_treino[0]
irrelevantes_brutos_treino = resultados_brutos_treino[1]
tweets_totais_treino = relevantes_brutos_treino + irrelevantes_brutos_treino
resultados_teste = test.Relevância.value_counts(True)
relevantes_teste = resultados_teste[0]
irrelevantes_teste = resultados_teste[1]
resultados_brutos_teste = test.Relevância.value_counts()
relevantes_brutos_teste = resultados_brutos_teste[0]
irrelevantes_brutos_teste = resultados_brutos_teste[1]
tweets_totais_teste = relevantes_brutos_teste + irrelevantes_brutos_teste

print('Tweets relevantes em Treinamento: {0}'.format(relevantes_brutos_treino))
print('Tweets irrelevantes em Treinamento: {0}'.format(irrelevantes_brutos_treino))
print('Tweets totais em Treinamento: {0}'.format(tweets_totais_treino))
print('Porcentagem de Tweets relevantes em Treinamento: {0:.2f}%'.format(relevantes_treino*100))
print('Porcentagem de Tweets irrelevantes em Treinamento: {0:.2f}%'.format(irrelevantes_treino*100))
print('Tweets relevantes em Teste: {0}'.format(relevantes_brutos_teste))
print('Tweets irrelevantes em Teste: {0}'.format(irrelevantes_brutos_teste))
print('Tweets totais em Teste: {0}'.format(tweets_totais_teste))
print('Porcentagem de Tweets relevantes em Teste: {0:.2f}%'.format(relevantes_teste*100))
print('Porcentagem de Tweets irrelevantes em Teste: {0:.2f}%'.format(irrelevantes_teste*100))

Tweets relevantes em Treinamento: 165
Tweets irrelevantes em Treinamento: 133
Tweets totais em Treinamento: 298
Porcentagem de Tweets relevantes em Treinamento: 55.37%
Porcentagem de Tweets irrelevantes em Treinamento: 44.63%
Tweets relevantes em Teste: 115
Tweets irrelevantes em Teste: 85
Tweets totais em Teste: 200
Porcentagem de Tweets relevantes em Teste: 57.50%
Porcentagem de Tweets irrelevantes em Teste: 42.50%


In [97]:
def classificador_de_tweets(data,words):
    p_possiveis,p_relevantes,p_irrelevantes = words
    classificacoes = []
    for index,row in data.iterrows():
        tweet = row.Teste
        prob_relevante,prob_irrelevante = 1,1
        for palavra in cleanup(tweet).split():
            prob_relevante *= (p_relevantes.get(palavra,0)+1)/(len(p_possiveis) + sum(p_relevantes.values()))
            prob_irrelevante *= (p_irrelevantes.get(palavra,0)+1)/(len(p_possiveis) + sum(p_irrelevantes.values()))
        classificacoes.append('Não relevante' if prob_relevante < prob_irrelevante else 'Relevante')
    return classificacoes

In [98]:
test['Classificação'] = classificador_de_tweets(test,p_count)
test.head(10)

,Teste,Relevância,Classificação
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante,Relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante,Relevante
2,quem ai quer me dar um ps4 ?,Não relevante,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante,Não relevante
5,@predatorium @robertadbrito tudo menos a bater...,Relevante,Relevante
6,@lucas_hunger aliás eu venderia o ps4 pra comp...,Relevante,Não relevante
7,@nctclia ps4 mano kkk,Não relevante,Não relevante
8,@xtheslime ok😔\n\n(esqueça seu ps4 que eu ia c...,Não relevante,Relevante
9,@robertadbrito ergonomia do de xbox é infinit...,Relevante,Relevante


In [99]:
pd.crosstab(test.Relevância,test.Classificação)

Classificação,Não relevante,Relevante
Relevância,,
Não relevante,70,45
Relevante,20,65


In [103]:
(pd.crosstab(test.Relevância,test.Classificação,normalize = 'index')*100).round(decimals=3)

Classificação,Não relevante,Relevante
Relevância,,
Não relevante,60.870,39.130
Relevante,23.529,76.471


In [95]:
lista_treino = []
for tweet in train.Treinamento:
    lista_treino.append(''.join(cleanup(tweet)))
for tweet in range(len(train.Treinamento)):
    train.Treinamento[tweet] = lista_treino[tweet]
test.head(10)

C:\Users\jonat\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Teste,Relevância,Classificação
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante,Relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante,Relevante
2,quem ai quer me dar um ps4 ?,Não relevante,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante,Não relevante
5,@predatorium @robertadbrito tudo menos a bater...,Relevante,Relevante
6,@lucas_hunger aliás eu venderia o ps4 pra comp...,Relevante,Não relevante
7,@nctclia ps4 mano kkk,Não relevante,Não relevante
8,@xtheslime ok😔\n\n(esqueça seu ps4 que eu ia c...,Não relevante,Relevante
9,@robertadbrito ergonomia do de xbox é infinit...,Relevante,Relevante


Pronto! Agora temos um dataframe sem os sinais de pontuação. Isso ajudará o nosso classificador a fazer a análise! A seguir iremos usar a função split para criar 2 listas com palavras: uma de irrelevantes e outra de relevantes!

In [37]:
relevantes = []         #lista com palavras que aparecem em tweets relevantes
irrelevantes = []       #lista com palavras que aparecem em tweets irrelevantes

#A função extend é parecida com o append, mas põe cada elemento em uma linha. Isso ajuda na hora de montar o dataframe
for tweet in range(len(train)):
    if train.Relevância[tweet] == 'Relevante':
        separador = train.Treinamento[tweet].split()
        relevantes.extend(separador)
    elif train.Relevância[tweet] == 'Não relevante':
        separador = train.Treinamento[tweet].split()
        irrelevantes.extend(separador)

In [38]:
df_relevantes = pd.DataFrame(relevantes)      #DataFrame com palavras relevantes
df_relevantes.columns = ['Número de vezes que aparece em tweets relevantes']
counts_relevantes = df_relevantes['Número de vezes que aparece em tweets relevantes'].value_counts()

df_irrelevantes = pd.DataFrame(irrelevantes)  #DataFrame com palavras irrelevantes
df_irrelevantes.columns = ['Número de vezes que aparece em tweets irrelevantes']
counts_irrelevantes = df_irrelevantes['Número de vezes que aparece em tweets irrelevantes'].value_counts()

In [39]:
counts_relevantes_df = pd.DataFrame(counts_relevantes)     #DataFrame com frequências absolutas de palavras relevantes
counts_irrelevantes_df = pd.DataFrame(counts_irrelevantes) #DataFrame com frequências absolutas de palavras irrelevantes

In [40]:
 #Junção dos 2 DataFrames, para facilitar a análise. O outer faz com que junte as 2 colunas, e não misture as linhas.
counts_df = counts_relevantes_df.join(counts_irrelevantes_df, how="outer")

In [41]:
filtro1 = counts_df['Número de vezes que aparece em tweets relevantes'].isnull()
counts_df.loc[filtro1,'Número de vezes que aparece em tweets relevantes'] = 0

In [42]:
filtro2 = counts_df['Número de vezes que aparece em tweets irrelevantes'].isnull()
counts_df.loc[filtro2,'Número de vezes que aparece em tweets irrelevantes'] = 0

In [43]:
counts_df.sort_values('Número de vezes que aparece em tweets relevantes', ascending=False).head(5)

,Número de vezes que aparece em tweets relevantes,Número de vezes que aparece em tweets irrelevantes
o,142.0,71.0
ps4,139.0,161.0
de,104.0,65.0
que,98.0,34.0
e,84.0,70.0


In [44]:
test.head(10)

,Teste,Relevância
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante
2,quem ai quer me dar um ps4 ?,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante
5,@predatorium @robertadbrito tudo menos a bater...,Relevante
6,@lucas_hunger aliás eu venderia o ps4 pra comp...,Relevante
7,@nctclia ps4 mano kkk,Não relevante
8,@xtheslime ok😔\n\n(esqueça seu ps4 que eu ia c...,Não relevante
9,@robertadbrito ergonomia do de xbox é infinit...,Relevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [45]:
def classificador_de_tweets(tweet):
    prob_relevante = 1
    prob_irrelevante = 1
    soma_relevantes = sum(counts_df['Número de vezes que aparece em tweets relevantes'])
    soma_irrelevantes = sum(counts_df['Número de vezes que aparece em tweets irrelevantes'])
    soma_total = len(counts_df)
    for palavra in cleanup(tweet):
        if palavra in counts_df.index:
            prob_relevante *= ((counts_df['Número de vezes que aparece em tweets relevantes'][palavra])/(soma_total+soma_relevantes))
            prob_irrelevante *= ((counts_df['Número de vezes que aparece em tweets irrelevantes'][palavra])/(soma_total+soma_irrelevantes))
        elif palavra not in counts_df.index:
            prob_irrelevante *= (1/(soma_total+soma_irrelevantes))
            prob_relevante *= (1/(soma_total+soma_irrelevantes))
    prob_relevante *= (relevantes_brutos_treino/tweets_totais_treino)
    prob_irrelevante *= (irrelevantes_brutos_treino/tweets_totais_treino)
           
    if prob_relevante > prob_irrelevante:
        return 'Relevante'
    else:
        return 'Não relevante'    

In [48]:
classificador_de_tweets('o ps4')

'Não relevante'

In [50]:
classificacoes = []
for tweet in test.Teste:
    classificacoes.append(classificador_de_tweets(tweet))
test['Classificador'] = classificacoes

In [52]:
test.head(10)

,Teste,Relevância,Classificador
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante,Não relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante,Não relevante
2,quem ai quer me dar um ps4 ?,Não relevante,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante,Não relevante
5,@predatorium @robertadbrito tudo menos a bater...,Relevante,Não relevante
6,@lucas_hunger aliás eu venderia o ps4 pra comp...,Relevante,Não relevante
7,@nctclia ps4 mano kkk,Não relevante,Não relevante
8,@xtheslime ok😔\n\n(esqueça seu ps4 que eu ia c...,Não relevante,Não relevante
9,@robertadbrito ergonomia do de xbox é infinit...,Relevante,Não relevante


In [53]:
pd.crosstab(test.Relevância,test.Classificador)

Classificador,Não relevante
Relevância,
Não relevante,115
Relevante,85


In [54]:
test.head(10)

,Teste,Relevância,Classificador
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,Não relevante,Não relevante
1,se isso influenciasse no preço do ps4/ps5 eu a...,Relevante,Não relevante
2,quem ai quer me dar um ps4 ?,Não relevante,Não relevante
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,Não relevante,Não relevante
4,@dudalbessa falando em tv\neu queria comprar u...,Relevante,Não relevante
5,@predatorium @robertadbrito tudo menos a bater...,Relevante,Não relevante
6,@lucas_hunger aliás eu venderia o ps4 pra comp...,Relevante,Não relevante
7,@nctclia ps4 mano kkk,Não relevante,Não relevante
8,@xtheslime ok😔\n\n(esqueça seu ps4 que eu ia c...,Não relevante,Não relevante
9,@robertadbrito ergonomia do de xbox é infinit...,Relevante,Não relevante


In [61]:
positivos_verdadeiros,positivos_falsos,negativos_verdadeiros,negativos_falsos = 0,0,0,0

for i in test.index:
    test.Relevância = classificador_de_tweets(test.Teste[i])
    if test.Relevância[i] == 'Relevante':
        if test.Relevância[i] == classificador_de_tweets(test.Teste[i]):
            positivos_verdadeiros += 1
        elif test.Relevância[i] != classificador_de_tweets(test.Teste[i]):
            positivos_falsos += 1
    elif test.Relevância[i] == 'Não relevante':
        if test.Relevância[i] == classificador_de_tweets(test.Teste[i]):
            negativos_verdadeiros += 1
        elif test.Relevância[i] != classificador_de_tweets(test.Teste[i]):
            negativos_falsos += 1
            
print(f'O classificador identificou {positivos_verdadeiros} tweets positivos verdadeiros, com {positivos_verdadeiros/2}% dos tweets.')
print(f'O classificador identificou {positivos_falsos} tweets positivos falsos, com {positivos_falsos/2}% dos tweets.')
print(f'O classificador identificou {negativos_verdadeiros} tweets negativos verdadeiros, com {negativos_verdadeiros/2}% dos tweets.')
print(f'O classificador identificou {negativos_falsos} tweets negativos falsos, com {negativos_falsos/2}% dos tweets.')

O classificador identificou 0 tweets positivos verdadeiros, com 0.0% dos tweets.
O classificador identificou 0 tweets positivos falsos, com 0.0% dos tweets.
O classificador identificou 200 tweets negativos verdadeiros, com 100.0% dos tweets.
O classificador identificou 0 tweets negativos falsos, com 0.0% dos tweets.


___
### Concluindo

Ao ver o fim 

As mensagens que contém sarcasmo não são classificadas pelo classificador como tais, pois o significado intendido pelo usuário não é compreendido. Isso acontece por conta do classificador apenas verificar se ele seria releveante ou não, e não analisar os significados reais da palavra. Mensagens com dupla negação tambem não são classificadas como tal pelo classificador, pois ele analisa a palavra individualmente e não a possição que ocupa no texto. Um plano de expansão incluiria um aumento significativo das palavras que seriam recolhida, para que exista a possibilidade de classificar os tweets de maneira mais precisa. Além disso, novas técnicas seriam pesquisadas e implementadas para tentar distinguir os tweets reais e os que tem dupla negação e sarcasmo.

O nosso projeto deveria ser financiado pois, com mais tempo de trabalho, poderíamos aperfeiçoar o nosso classificador para entender mensagens de sarcasmo, dupla negação, etc. Além disso, criaríamos mais categorias (como por exemplo, a opção "neutro") para tornar o classificador mais preciso. Em mais tempo de trabalho, seria possível usar esse classificador em tempo real!

Não seria possível usar alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets, pois os tweets para treinamento servem para ensinar o computador como ler as mensagens relevantes e irrelevantes. 

O classificador de Naive Bayes poderia ser usado para vários outros casos, como por exemplo recomendar um filme ou uma música para um usuário, dependendo do que ele vai selecionando. Assim, o classificador calcularia o filme ou música com a maior probabilidade de ser aprovada, levando em conta questões como gênero, duração, atores envolvidos, estilo da música, cantor/banda, etc.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**